In [1]:
!pip install -U \
torch==2.9.0+cu128 torchvision==0.24.0+cu128 torchaudio==2.9.0+cu128 \
transformers==5.0.0 accelerate==1.12.0 datasets==4.5.0 evaluate==0.4.3 \
peft==0.18.1 trl==0.27.2 bitsandbytes==0.49.1 \
huggingface-hub==1.3.7 tokenizers==0.22.2 \
sqlite-utils==3.38 sqlalchemy==2.0.30

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 530.9/530.9 kB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 122.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 46.7 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.46
    Uninstalling SQLAlchemy-2.0.46:
      Successfully uninstalled SQLAlchemy-2.0.46
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: huggingface

In [2]:
!pip list | grep -E 'transformers|accelerate|datasets|sqlalchemy|sqlite-utils|moz-sql-parser|evaluate|torch|torchvision'

accelerate                               1.12.0
datasets                                 4.5.0
evaluate                                 0.4.3
sentence-transformers                    5.2.2
sqlalchemy-spanner                       1.17.2
sqlite-utils                             3.38
tensorflow-datasets                      4.9.9
torch                                    2.9.0+cu128
torchao                                  0.10.0
torchaudio                               2.9.0+cu128
torchcodec                               0.8.0+cu128
torchdata                                0.11.0
torchsummary                             1.5.1
torchtune                                0.6.1
torchvision                              0.24.0+cu128
transformers                             5.0.0
vega-datasets                            0.9.0


In [3]:
%%bash
python - <<'PY'
import random, numpy as np, torch, os
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
print("Seeds fixed to", SEED)
PY


Seeds fixed to 42


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from huggingface_hub import login
from google.colab import userdata

# Retrieve the Hugging Face API key from Colab's secrets manager
hf_token = userdata.get('HF_API_KEY')

# Log in to Hugging Face Hub using the retrieved token
login(hf_token)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from pathlib import Path
import json

DATA_DIR = Path("/content/drive/MyDrive/chichewa-text2sql/data")

train_path = DATA_DIR / "train.json"
dev_path   = DATA_DIR / "dev.json"
test_path  = DATA_DIR / "test.json"

print("Train exists:", train_path.exists())
print("Dev exists:", dev_path.exists())
print("Test exists:", test_path.exists())


Train exists: True
Dev exists: True
Test exists: True


In [7]:
def load_json(path: Path):
    with path.open("r", encoding="utf-8") as f:
        return json.load(f)


train_data = load_json(train_path)
dev_data   = load_json(dev_path)
test_data  = load_json(test_path)

print(f"Train size: {len(train_data)}")
print(f"Dev size:   {len(dev_data)}")
print(f"Test size:  {len(test_data)}")

Train size: 280
Dev size:   60
Test size:  60


In [8]:
# reflect every .sqlite file to create a plain-text schema description

import sqlalchemy as sa

DB_PATH = Path("/content/drive/MyDrive/chichewa-text2sql/data/database/chichewa_text2sql.db")

def get_schema_string() -> str:
    """Return a compact textual schema for the given database."""

    engine  = sa.create_engine(f"sqlite:///{DB_PATH}")
    insp    = sa.inspect(engine)

    parts   = []
    for tbl in sorted(insp.get_table_names()):
        cols = [c["name"] for c in insp.get_columns(tbl)]
        parts.append(f"{tbl}({', '.join(cols)})")

    schema_str = ", ".join(parts)
    return schema_str

In [9]:
schema_str = get_schema_string()
print(schema_str)

commodity_prices(id, add_name, epa_name, district, market, month_name, year, commodity, price, collection_date), food_insecurity(id, district, analyzed_population, time_period, percentage_population, insecurity_level, insecurity_desc_short, insecurity_desc_long), mse_daily(id, counter_id, ticker, trade_date, print_time, company_name, sector, high_price, low_price, bid_price, ask_price, previous_close_price, close_price, volume, dividend_mwk, dividend_yield_pct, earnings_yield_pct, pe_ratio, pbv_ratio, market_cap_mwk_mn, profit_after_tax_mwk_mn, shares_outstanding), population(id, region_name, region_code, admin_status, district_code, ea_number, ea_code, ta_code, ta_name, population_male, population_female, number_households, district_name, total_population), production(id, district, crop, yield, season)


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "defog/sqlcoder-7b-2"  # use preferred model
tokenizer  = AutoTokenizer.from_pretrained(model_name)
model      = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)
model.eval()

In [13]:
# ──────────────────────────────────────────────────────────────────────────
# 0 ▪ global seed (reproducible shuffles, torch, numpy, random, etc.)
# ──────────────────────────────────────────────────────────────────────────
import random, numpy as np, torch, os, gc
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)

# ──────────────────────────────────────────────────────────────────────────
# 1 ▪ zero-shot prompt builder (no demos)
# ──────────────────────────────────────────────────────────────────────────
def build_prompt_zero(nl_question: str, target_schema: str) -> str:
    return (
        "### Instruction:\n"
        "You are an expert SQL developer. Given the database schema and the "
        "question, return ONE valid SQL statement — output ONLY the SQL.\n\n"
        f"### Database Schema:\n{target_schema}\n\n"
        f"### Question:\n{nl_question}\n\n"
        "### SQL:\n"
    )

# ──────────────────────────────────────────────────────────────────────────
# 2 ▪ random-5-shot builder
# ──────────────────────────────────────────────────────────────────────────
NUM_SHOTS = 5

# Convert train_data to a Hugging Face Dataset object to use .shuffle() and .select()
from datasets import Dataset
train_dataset = Dataset.from_list(train_data)

DEMO_SET = [
    {**ex, "schema_str": get_schema_string()}
    for ex in train_dataset.shuffle(seed=SEED).select(range(NUM_SHOTS))
]

def build_prompt_random5(nl_question: str, target_schema: str) -> str:
    parts = ["### Instruction:\nReturn ONE SQL query only, based on the examples provided.\n"]
    for i, ex in enumerate(DEMO_SET, 1):
        parts += [
            f"### Example {i} Schema:\n{ex['schema_str']}",
            f"### Example {i} Question:\n{ex['question_en']}",
            f"### Example {i} SQL:\n{ex['sql_statement'].strip()}",
        ]
    parts += [
        f"### Database Schema:\n{target_schema}",
        f"### Question:\n{nl_question}",
        "### SQL:\n",
    ]
    return "\n".join(parts)

# ──────────────────────────────────────────────────────────────────────────
# 3 ▪ retrieved-5-shot builder (SBERT nearest neighbours)
# ──────────────────────────────────────────────────────────────────────────
from sentence_transformers import SentenceTransformer, util
embedder   = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
train_emb  = embedder.encode(train_dataset["question_en"], convert_to_tensor=True)

def get_k_shots(nl_question: str, k: int = 5):
    q_emb  = embedder.encode(nl_question, convert_to_tensor=True)
    hits   = util.semantic_search(q_emb, train_emb, top_k=k)[0]
    demos  = []
    for h in hits:
        ex = train_dataset[int(h["corpus_id"])]
        demos.append({
            "question"   : ex["question_en"],
            "query"      : ex["sql_statement"],
            "schema_str" : get_schema_string(),
        })
    return demos

def build_prompt_retrieved5(nl_question: str, target_schema: str) -> str:
    demos = get_k_shots(nl_question, 5)
    parts = ["### Instruction:\nReturn ONE SQL query only, based on the examples provided.\n"]
    for i, ex in enumerate(demos, 1):
        parts += [
            f"### Example {i} Schema:\n{ex['schema_str']}",
            f"### Example {i} Question:\n{ex['question']}",
            f"### Example {i} SQL:\n{ex['query'].strip()}",
            "### End\n",
        ]
    parts += [
        f"### Database Schema:\n{target_schema}",
        f"### Question:\n{nl_question}",
        "### SQL:\n",
    ]
    return "\n".join(parts)

# ──────────────────────────────────────────────────────────────────────────
# 4 ▪ shared generate_sql that accepts a *builder* argument
# ──────────────────────────────────────────────────────────────────────────
import re

def make_generator(builder_fn, tokenizer, model): # Modified to accept tokenizer and model
    def _gen(nl_question: str, schema: str) -> str:
        prompt = builder_fn(nl_question, schema)
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        input_len = inputs["input_ids"].shape[1]

        model.config.pad_token_id = tokenizer.eos_token_id
        out = model.generate(
            **inputs,
            max_new_tokens=128,
            num_beams=3,
            early_stopping=True,
            do_sample=False,
        )
        gen_text = tokenizer.decode(out[0, input_len:], skip_special_tokens=True)
        sql = re.split(r"(###|\n\s*\n|```)", gen_text, 1)[0]
        sql = sql.split(";")[0].replace("\n", " ").strip()
        return sql.split(";", 1)[0].strip()
    return _gen

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/526 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# choose the prompt mode here ─────────────────────────────────────────────
PROMPT_MODE = 0          # 0 = zero-shot, 1 = random-5, 2 = retrieved-5
builder = [build_prompt_zero,
           build_prompt_random5,
           build_prompt_retrieved5][PROMPT_MODE]
generate_sql = make_generator(builder)

In [ ]:
example      = test_data[2]
display(example)
print("=="*50 + "\n")
schema_str   = get_schema_string()
predicted_sql= generate_sql(example["question_en"], schema_str)
print("Gold SQL:", example["sql_statement"])
print("Pred SQL:", predicted_sql)

In [ ]:
test_data[2]

{'id': 11,
 'question_en': 'What was the yield of wheat in nkhotakota?',
 'question_ny': 'Ndi tiligu ochuluka bwanj adakololedwa ku Nkhotakota?',
 'sql_statement': "SELECT Yield FROM production WHERE District = 'Nkhotakota' AND Crop = 'Wheat';",
 'sql_result': '[(0.0,)]',
 'difficulty_level': 'easy',
 'table': 'production'}

In [17]:
import re
_ws   = re.compile(r"\s+")
_comm = re.compile(r"\s*,\s*")          # blank(s) ↔ comma ↔ blank(s)

def exact_match(pred: str, gold: str) -> bool:
    def norm(s: str) -> str:
        s = s.strip().lower()
        s = _ws.sub(" ", s)             # collapse runs of whitespace
        s = _comm.sub(", ", s)          # canonical “comma␣”
        s = s.rstrip(';')                # Remove trailing semicolon
        return s
    return norm(pred) == norm(gold)

In [18]:
from sqlglot import parse_one, expressions

def flatten_ast(node):
    """
    Recursively collect all node‐type names and literal values as lowercase strings.
    """
    out = set()

    def walk(n):
        # record the AST node type
        out.add(type(n).__name__.lower())

        # record any literal (e.g. identifiers, strings, numbers)
        if hasattr(n, "this") and isinstance(n.this, (str, int, float)):
            out.add(str(n.this).lower())

        # recurse into child expressions
        for arg in n.args.values():
            if isinstance(arg, list):
                for child in arg:
                    if isinstance(child, expressions.Expression):
                        walk(child)
            elif isinstance(arg, expressions.Expression):
                walk(arg)

    walk(node)
    return out

def component_match(pred_sql, gold_sql):
    try:
        pred_ast = parse_one(pred_sql)
        gold_ast = parse_one(gold_sql)
    except Exception:
        return 0.0

    pred_set = flatten_ast(pred_ast)
    gold_set = flatten_ast(gold_ast)
    if not gold_set:
        return 0.0
    return len(pred_set & gold_set) / len(gold_set)


In [19]:
import sqlite3, pandas as pd, numpy as np
from tqdm import tqdm
from datasets import Dataset

def run_query(sql: str, db_path: Path):
    """Return query result as a sorted list of tuples (order-independent)."""
    try:
        with sqlite3.connect(db_path) as conn:
            df = pd.read_sql_query(sql, conn)
        # sort rows + cols for order-invariant comparison
        return tuple(map(tuple, df.sort_index(axis=1).sort_values(list(df.columns)).to_numpy()))
    except Exception as e:

        return f"ERROR-{e}"

def execution_accuracy(dataset):
    """Compute Spider-style Execution Accuracy on the test split"""
    correct = 0
    for ex in tqdm(dataset, desc="Evaluating"):

        schema  = schema_str

        pred_sql= generate_sql(ex["question_en"], schema)

        # Use global DB_PATH
        gold    = run_query(ex["sql_statement"], DB_PATH)
        pred    = run_query(pred_sql, DB_PATH)

        if gold == pred:
            correct += 1

    return correct / len(dataset)

In [ ]:
example      = test_data[2]
schema_str   = get_schema_string()
predicted_sql= generate_sql(example["question_en"], schema_str)
print("Gold SQL:", example["sql_statement"])
print("Pred SQL:", predicted_sql)

In [ ]:
exact_match(predicted_sql, example["sql_statement"])

False

In [ ]:
component_match(predicted_sql, example["sql_statement"])

1.0

# Zero-shot

In [ ]:

import time, torch, numpy as np
from tqdm import tqdm
import warnings
from transformers import logging
import random
from datasets import Dataset # Import Dataset here

random.seed(42)

logging.set_verbosity_error()
warnings.filterwarnings('ignore')

# choose the prompt mode here ─────────────────────────────────────────────
PROMPT_MODE = 0          # 0 = zero-shot, 1 = random-5, 2 = retrieved-5
builder = [build_prompt_zero,
           build_prompt_random5,
           build_prompt_retrieved5][PROMPT_MODE]
generate_sql = make_generator(builder)

# 1)
# Use test_data as the evaluation dataset
# Ensure it's converted to a Hugging Face Dataset for shuffle/select methods
test_dataset = Dataset.from_list(test_data)
num_samples = len(test_dataset)
sample_test  = test_dataset.shuffle(seed=42).select(range(num_samples))


# 2) containers
em_scores, cm_scores, ex_scores, times = [], [], [], []

# 3) start fresh CUDA-peak tracking
if torch.cuda.is_available():
    torch.cuda.reset_peak_memory_stats()

# 4) main loop
for ex in tqdm(sample_test, desc="Evaluating"):
    # db_id = ex["db_id"] # Not used since get_schema_string does not take db_id
    # schema = get_schema_string(db_id) # Incorrect, get_schema_string takes no args
    schema = schema_str # Use the globally defined schema_str

    t0      = time.perf_counter()
    pred_sql= generate_sql(ex["question_en"], schema)
    times.append(time.perf_counter() - t0)

    gold_sql= ex["sql_statement"]

    # exact + component match
    em_scores.append( float(exact_match(pred_sql, gold_sql)) )
    cm_scores.append( component_match(pred_sql, gold_sql) )

    # execution accuracy
    gold_res= run_query(gold_sql,  DB_PATH)
    pred_res= run_query(pred_sql, DB_PATH)
    ex_scores.append( int(gold_res == pred_res) )

# 5) aggregate & report
print("\n\nExact Match        :", np.mean(em_scores))
print("Component Match    :", np.mean(cm_scores))
print("Execution Accuracy :", np.mean(ex_scores))
print("Avg. Latency  (s)  :", np.mean(times))
print("95% Latency  (s)   :", np.percentile(times, 95))
if torch.cuda.is_available():
    print("GPU Mem Peak       :", torch.cuda.max_memory_allocated() / 1e9, "GB")
else:
    print("GPU Mem Peak       : N/A (CPU)")

Evaluating: 100%|██████████| 60/60 [02:08<00:00,  2.15s/it]



Exact Match        : 0.0
Component Match    : 0.8517431748391501
Execution Accuracy : 0.13333333333333333
Avg. Latency  (s)  : 2.1357557217333465
95% Latency  (s)   : 3.4427179628499855
GPU Mem Peak       : 14.750436864 GB


# Few-shot (Random Sampling)

In [ ]:
# Evaluation on a fixed 20% Spider dev sample
import time, torch, numpy as np
from tqdm import tqdm
import warnings
from transformers import logging
import random
from datasets import Dataset # Import Dataset

random.seed(42)

logging.set_verbosity_error()
warnings.filterwarnings('ignore')

# choose the prompt mode here ─────────────────────────────────────────────
PROMPT_MODE = 1          # 0 = zero-shot, 1 = random-5, 2 = retrieved-5
builder = [build_prompt_zero,
           build_prompt_random5,
           build_prompt_retrieved5][PROMPT_MODE]
generate_sql = make_generator(builder)

# 1)
# Use test_data for evaluation
test_dataset = Dataset.from_list(test_data)
num_samples = len(test_dataset)
sample_test  = test_dataset.shuffle(seed=42).select(range(num_samples))


# 2) containers
em_scores, cm_scores, ex_scores, times = [], [], [], []

# 3) start fresh CUDA-peak tracking
if torch.cuda.is_available():
    torch.cuda.reset_peak_memory_stats()

# 4) main loop
for ex in tqdm(sample_test, desc="Evaluating"):
    # Use global schema_str
    schema  = schema_str

    t0      = time.perf_counter()
    pred_sql= generate_sql(ex["question_en"], schema) # Corrected to question_en
    times.append(time.perf_counter() - t0)

    gold_sql= ex["sql_statement"] # Corrected to sql_statement

    # exact + component match
    em_scores.append( float(exact_match(pred_sql, gold_sql)) )
    cm_scores.append( component_match(pred_sql, gold_sql) )

    # execution accuracy
    gold_res= run_query(gold_sql,  DB_PATH) # Use global DB_PATH
    pred_res= run_query(pred_sql, DB_PATH) # Use global DB_PATH
    ex_scores.append( int(gold_res == pred_res) )

# 5) aggregate & report
print("\n\nExact Match        :", np.mean(em_scores))
print("Component Match    :", np.mean(cm_scores))
print("Execution Accuracy :", np.mean(ex_scores))
print("Avg. Latency  (s)  :", np.mean(times))
print("95% Latency  (s)   :", np.percentile(times, 95))
if torch.cuda.is_available():
    print("GPU Mem Peak       :", torch.cuda.max_memory_allocated() / 1e9, "GB")
else:
    print("GPU Mem Peak       : N/A (CPU)")

Evaluating: 100%|██████████| 60/60 [02:27<00:00,  2.46s/it]



Exact Match        : 0.0
Component Match    : 0.9123081811262926
Execution Accuracy : 0.45
Avg. Latency  (s)  : 2.446125197216664
95% Latency  (s)   : 3.568981374050014
GPU Mem Peak       : 17.903938048 GB


# Few-shot (Retrieval-Augmented)

In [ ]:
# Evaluation on a fixed 20% Spider dev sample
import time, torch, numpy as np
from tqdm import tqdm
import warnings
from transformers import logging
import random
from datasets import Dataset

random.seed(42)

logging.set_verbosity_error()
warnings.filterwarnings('ignore')

# choose the prompt mode here ─────────────────────────────────────────────
PROMPT_MODE = 2          # 0 = zero-shot, 1 = random-5, 2 = retrieved-5
builder = [build_prompt_zero,
           build_prompt_random5,
           build_prompt_retrieved5][PROMPT_MODE]
generate_sql = make_generator(builder)

# 1)
# Use test_data for evaluation
test_dataset = Dataset.from_list(test_data)
num_samples = len(test_dataset)
sample_test  = test_dataset.shuffle(seed=42).select(range(num_samples))


# 2) containers
em_scores, cm_scores, ex_scores, times = [], [], [], []

# 3) start fresh CUDA-peak tracking
if torch.cuda.is_available():
    torch.cuda.reset_peak_memory_stats()

# 4) main loop
for ex in tqdm(sample_test, desc="Evaluating"):
    # Use global schema_str
    schema  = schema_str

    t0      = time.perf_counter()
    pred_sql= generate_sql(ex["question_en"], schema)
    times.append(time.perf_counter() - t0)

    gold_sql= ex["sql_statement"]

    # exact + component match
    em_scores.append( float(exact_match(pred_sql, gold_sql)) )
    cm_scores.append( component_match(pred_sql, gold_sql) )

    # execution accuracy
    gold_res= run_query(gold_sql,  DB_PATH) # Use global DB_PATH
    pred_res= run_query(pred_sql, DB_PATH) # Use global DB_PATH
    ex_scores.append( int(gold_res == pred_res) )

# 5) aggregate & report
print("\n\nExact Match        :", np.mean(em_scores))
print("Component Match    :", np.mean(cm_scores))
print("Execution Accuracy :", np.mean(ex_scores))
print("Avg. Latency  (s)  :", np.mean(times))
print("95% Latency  (s)   :", np.percentile(times, 95))
if torch.cuda.is_available():
    print("GPU Mem Peak       :", torch.cuda.max_memory_allocated() / 1e9, "GB")
else:
    print("GPU Mem Peak       : N/A (CPU)")

Evaluating: 100%|██████████| 60/60 [02:32<00:00,  2.53s/it]



Exact Match        : 0.0
Component Match    : 0.9478143041169357
Execution Accuracy : 0.6333333333333333
Avg. Latency  (s)  : 2.519987743216677
95% Latency  (s)   : 4.405547950499896
GPU Mem Peak       : 18.465375232 GB


# Comprehensive evaluation across all 5 Models


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Define the dictionary of models
MODELS = {
    "sqlcoder_7b"        : "defog/sqlcoder-7b-2",
    "deepseek_coder_inst": "deepseek-ai/deepseek-coder-6.7b-instruct",
    "llama3_8b_inst"     : "meta-llama/Llama-3.1-8B-Instruct",
    "bloomz_7b_mt"       : "bigscience/bloomz-7b1-mt",
    "codellama_7b_inst"  : "codellama/CodeLlama-7b-Instruct-hf"
}

def load_model(model_path: str):
    """Loads the tokenizer and model from HuggingFace."""
    print(f"Loading model: {model_path}...")
    tokenizer  = AutoTokenizer.from_pretrained(model_path)
    model      = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map="auto",
        torch_dtype=torch.float16
    )
    model.eval()
    print(f"Model {model_path} loaded successfully.")
    return tokenizer, model

In [10]:
import gc
from pathlib import Path

RESULTS_DIR = Path('/content/drive/MyDrive/chichewa-text2sql/results')

if RESULTS_DIR.exists():
    print(f"Results directory already exists: {RESULTS_DIR}")
else:
    RESULTS_DIR.mkdir(parents=True, exist_ok=True)
    print(f"Results directory created: {RESULTS_DIR}")

all_results = []
print("Initialized 'all_results' list.")


Results directory created: /content/drive/MyDrive/chichewa-text2sql/results
Initialized 'all_results' list.


## Run comprehensive evaluation

### Subtask:
Iterate through each model in the `MODELS` dictionary, load it, then evaluate its performance across zero-shot, random-5-shot, and retrieved-5-shot prompt strategies using `test_data`. Calculate Exact Match, Component Match, Execution Accuracy, and latency metrics for each strategy. Store these metrics in the `all_results` list. After evaluating each model, deallocate its resources (model, tokenizer), clear the CUDA cache, and run garbage collection.


**Reasoning**:
The subtask requires iterating through each model and prompt strategy to perform evaluation, calculate metrics, and store the results. This will be implemented in a single code block to manage resources and maintain the execution flow.



In [22]:
import time, torch, numpy as np
from tqdm import tqdm
import warnings
from transformers import logging
import random
from datasets import Dataset # Import Dataset
import gc # Import garbage collector

random.seed(42)

logging.set_verbosity_error()
warnings.filterwarnings('ignore')

# 1. Initialize test_dataset and sample_test
test_dataset = Dataset.from_list(test_data)
num_samples = len(test_dataset)
sample_test  = test_dataset.shuffle(seed=42).select(range(num_samples))

# 2. Define prompt_builders
prompt_builders = [build_prompt_zero,
                   build_prompt_random5,
                   build_prompt_retrieved5]

# 3. Define prompt_strategy_names
prompt_strategy_names = ["zero-shot", "random-5-shot", "retrieved-5-shot"]

# 4. Loop through each model_name and model_path in the MODELS dictionary
for model_name, model_path in MODELS.items():
    # 5. Print a header indicating the current model
    print(f"\n{'='*80}\nEvaluating Model: {model_name} ({model_path})\n{'='*80}")

    # 6. Load the tokenizer and model for the current model_path
    tokenizer, model = load_model(model_path)

    # 7. Loop through the PROMPT_MODE from 0 to 2
    for PROMPT_MODE in range(len(prompt_builders)):
        # 8. Get strategy_name and print a header
        strategy_name = prompt_strategy_names[PROMPT_MODE]
        print(f"\n{'-'*60}\n  Evaluating Strategy: {strategy_name}\n{'-'*60}")

        # 9. Select the appropriate builder function
        builder = prompt_builders[PROMPT_MODE]

        # 10. Create the generate_sql function
        generate_sql = make_generator(builder, tokenizer, model)

        # 11. Initialize empty lists
        em_scores, cm_scores, ex_scores, times = [], [], [], []

        # 12. Start fresh CUDA-peak tracking
        if torch.cuda.is_available():
            torch.cuda.reset_peak_memory_stats()

        # 13. Iterate through each ex (example) in sample_test
        for ex in tqdm(sample_test, desc=f"Evaluating {model_name} - {strategy_name}"):
            # 14. Get the global schema_str
            schema  = schema_str

            # 15. Record the start time
            t0      = time.perf_counter()

            # 16. Generate the pred_sql
            pred_sql= generate_sql(ex["question_en"], schema)

            # 17. Record the end time and append the elapsed time
            times.append(time.perf_counter() - t0)

            # 18. Get the gold_sql
            gold_sql= ex["sql_statement"]

            # 19. Calculate Exact Match
            em_scores.append( float(exact_match(pred_sql, gold_sql)) )

            # 20. Calculate Component Match
            cm_scores.append( component_match(pred_sql, gold_sql) )

            # 21. Execute both gold_sql and pred_sql
            gold_res= run_query(gold_sql,  DB_PATH)
            pred_res= run_query(pred_sql, DB_PATH)

            # 22. Append execution accuracy score
            ex_scores.append( int(gold_res == pred_res) )

        # 23. After the inner loop, calculate the mean metrics
        mean_em = np.mean(em_scores)
        mean_cm = np.mean(cm_scores)
        mean_ex = np.mean(ex_scores)
        mean_latency = np.mean(times)
        p95_latency = np.percentile(times, 95)

        # 24. Determine gpu_mem_peak
        gpu_mem_peak = "N/A (CPU)"
        if torch.cuda.is_available():
            gpu_mem_peak = f"{torch.cuda.max_memory_allocated() / 1e9:.3f} GB"

        # 25. Print all calculated metrics
        print(f"\nExact Match        : {mean_em:.4f}")
        print(f"Component Match    : {mean_cm:.4f}")
        print(f"Execution Accuracy : {mean_ex:.4f}")
        print(f"Avg. Latency  (s)  : {mean_latency:.4f}")
        print(f"95% Latency  (s)   : {p95_latency:.4f}")
        print(f"GPU Mem Peak       : {gpu_mem_peak}")

        # 26. Append results to all_results list
        all_results.append({
            "model_name": model_name,
            "prompt_strategy": strategy_name,
            "exact_match": mean_em,
            "component_match": mean_cm,
            "execution_accuracy": mean_ex,
            "avg_latency_s": mean_latency,
            "p95_latency_s": p95_latency,
            "gpu_mem_peak": gpu_mem_peak
        })

    # 27. After the prompt strategy loop, deallocate resources
    del model
    del tokenizer
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()
    # 28. Print deallocation message
    print(f"\nResources deallocated for {model_name}.")

print(f"\n{'='*80}\nAll evaluations complete.\nFinal results stored in 'all_results' list.\n{'='*80}")


Evaluating Model: sqlcoder_7b (defog/sqlcoder-7b-2)
Loading model: defog/sqlcoder-7b-2...


config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

...


Exact Match        : 0.1333
Component Match    : 0.8346
Execution Accuracy : 0.2000
Avg. Latency  (s)  : 3.0887
95% Latency  (s)   : 5.2452
GPU Mem Peak       : 14.734 GB

------------------------------------------------------------
  Evaluating Strategy: random-5-shot
------------------------------------------------------------


Evaluating codellama_7b_inst - random-5-shot: 100%|██████████| 60/60 [05:23<00:00,  5.39s/it]



Exact Match        : 0.3833
Component Match    : 0.9145
Execution Accuracy : 0.4833
Avg. Latency  (s)  : 5.3783
95% Latency  (s)   : 7.5031
GPU Mem Peak       : 17.901 GB

------------------------------------------------------------
  Evaluating Strategy: retrieved-5-shot
------------------------------------------------------------


Evaluating codellama_7b_inst - retrieved-5-shot: 100%|██████████| 60/60 [04:07<00:00,  4.13s/it]



Exact Match        : 0.6833
Component Match    : 0.9588
Execution Accuracy : 0.6833
Avg. Latency  (s)  : 4.1138
95% Latency  (s)   : 7.6885
GPU Mem Peak       : 18.464 GB

Resources deallocated for codellama_7b_inst.

All evaluations complete.
Final results stored in 'all_results' list.


**Reasoning**:
The comprehensive evaluation across all models and prompt strategies has been completed and the results are stored in the 'all_results' list. The next logical step is to display these collected results in a readable format, such as a pretty JSON.



In [24]:
import json

# Convert numpy types to native Python types for JSON serialization
def convert_numpy_types(obj):
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {k: convert_numpy_types(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_numpy_types(elem) for elem in obj]
    return obj

# Apply conversion to all_results before printing as JSON
serializable_results = [convert_numpy_types(res) for res in all_results]

print(json.dumps(serializable_results, indent=2))

[
  {
    "model_name": "sqlcoder_7b",
    "prompt_strategy": "zero-shot",
    "exact_match": 0.0,
    "component_match": 0.8517431748391501,
    "execution_accuracy": 0.13333333333333333,
    "avg_latency_s": 2.10087970278333,
    "p95_latency_s": 3.3589271491499852,
    "gpu_mem_peak": "14.750 GB"
  },
  {
    "model_name": "sqlcoder_7b",
    "prompt_strategy": "random-5-shot",
    "exact_match": 0.31666666666666665,
    "component_match": 0.9123081811262926,
    "execution_accuracy": 0.45,
    "avg_latency_s": 2.417681345316676,
    "p95_latency_s": 3.4922735174998247,
    "gpu_mem_peak": "17.904 GB"
  },
  {
    "model_name": "sqlcoder_7b",
    "prompt_strategy": "retrieved-5-shot",
    "exact_match": 0.5666666666666667,
    "component_match": 0.9478143041169357,
    "execution_accuracy": 0.6333333333333333,
    "avg_latency_s": 2.481812889283295,
    "p95_latency_s": 4.309542341399832,
    "gpu_mem_peak": "18.465 GB"
  },
  {
    "model_name": "deepseek_coder_inst",
    "prompt_st

## Save evaluation results

### Subtask:
Convert the `all_results` list into a pandas DataFrame and save it as a CSV file named `evaluation_results.csv` in the `/content/drive/MyDrive/chichewa-text2sql/results` directory.


## Summary:

### Q&A
The task involved evaluating multiple large language models for text-to-SQL generation across different prompting strategies to determine their performance.

### Data Analysis Key Findings
*   **Retrieved-5-shot prompting consistently outperformed other strategies**: For nearly all models, the "retrieved-5-shot" strategy yielded the highest Exact Match, Component Match, and Execution Accuracy scores.
    *   `llama3_8b_inst` achieved the highest Exact Match (0.6833), Component Match (0.9566), and Execution Accuracy (0.7) with "retrieved-5-shot" prompting.
    *   `codellama_7b_inst` also showed strong performance with "retrieved-5-shot", matching `llama3_8b_inst` in Exact Match (0.6833) and Execution Accuracy (0.6833), and achieving the highest Component Match (0.9588).
*   **Zero-shot prompting generally performed poorly**: "Zero-shot" strategies consistently resulted in the lowest Exact Match, Component Match, and Execution Accuracy scores across all evaluated models. For instance, `sqlcoder_7b`, `llama3_8b_inst`, and `bloomz_7b_mt` all had Exact Match scores of 0.0 or close to 0.0 in the zero-shot setting.
*   **Latency varied by model and strategy**:
    *   `bloomz_7b_mt` generally had the lowest average latency across all strategies (e.g., 1.118s for zero-shot, 1.928s for retrieved-5-shot).
    *   `deepseek_coder_inst` exhibited the highest average latencies, reaching up to 7.41s with "retrieved-5-shot" prompting.
*   **GPU memory usage increased with more complex prompting**: Peak GPU memory consumption was generally higher for "random-5-shot" and "retrieved-5-shot" strategies compared to "zero-shot", indicating the increased memory load from processing more examples in the prompt.

### Insights or Next Steps
*   For optimal text-to-SQL performance, prioritize "retrieved-5-shot" prompting over "zero-shot" or "random-5-shot" methods, even considering its slightly higher latency and GPU memory usage.
*   Investigate the trade-off between performance gains and resource consumption (latency, GPU memory) for `bloomz_7b_mt` given its lower latency but also lower accuracy, and `deepseek_coder_inst` / `llama3_8b_inst` / `codellama_7b_inst` for their higher accuracy but also higher resource consumption.


# Task
Save the `all_results` list to a JSON file named `zero_and_few_shot_english_results.json` and a CSV file named `zero_and_few_shot_english_results.csv` in the `RESULTS_DIR` directory.

## Save evaluation results as JSON and CSV

### Subtask:
Convert the `all_results` list into a serializable format for JSON, save it as `zero_and_few_shot_english_results.json` in the results directory, then convert it to a pandas DataFrame and save as `zero_and_few_shot_english_results.csv`.


## Summary:

### Data Analysis Key Findings
- The evaluation results, stored in the `all_results` list, were successfully saved to two different file formats for persistence and ease of access.
- A JSON file named `zero_and_few_shot_english_results.json` was created in the `RESULTS_DIR` to store the structured evaluation data.
- A CSV file named `zero_and_few_shot_english_results.csv` was also generated in the `RESULTS_DIR`, providing a tabular representation of the same evaluation results, suitable for spreadsheet applications or further data analysis.

### Insights or Next Steps
- The saved JSON and CSV files provide a durable record of the zero-shot and few-shot English evaluation results, enabling future analysis, reporting, or integration into other systems.
- These files can now be easily shared with stakeholders or used as input for visualization tools to present the model's performance in a comprehensive manner.


In [25]:
import json
import pandas as pd

# Define file names
json_file_name = "zero_and_few_shot_english_results.json"
csv_file_name = "zero_and_few_shot_english_results.csv"

# Construct full paths
json_file_path = RESULTS_DIR / json_file_name
csv_file_path  = RESULTS_DIR / csv_file_name

# 1. Save results to JSON file
with open(json_file_path, "w", encoding="utf-8") as f:
    json.dump(serializable_results, f, indent=2)
print(f"Evaluation results saved to JSON: {json_file_path}")

# 2. Save results to CSV file
# It's better to create a DataFrame directly from all_results as it preserves numpy types
# until pandas handles them, or use serializable_results if type conversion is already done.
results_df = pd.DataFrame(serializable_results)
results_df.to_csv(csv_file_path, index=False)
print(f"Evaluation results saved to CSV: {csv_file_path}")

Evaluation results saved to JSON: /content/drive/MyDrive/chichewa-text2sql/results/zero_and_few_shot_english_results.json
Evaluation results saved to CSV: /content/drive/MyDrive/chichewa-text2sql/results/zero_and_few_shot_english_results.csv
